In [64]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import joblib
import pandas as pd
import seaborn as sns
from urllib.request import urlretrieve
import re
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.naive_bayes import MultinomialNB

In [2]:
table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

df = pd.read_parquet('metadata.parquet')

In [2]:
df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547


In [7]:
harm_dict = {
    "porn": [10, 0, 1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [5, 0, 1],
    "creampie": [6, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "gore": [8, 1, 0],
    "hentai": [10, 0, 1],
    "lolicon": [10, 0, 1],
    "moaning": [5, 0, 1],
    "nsfw": [8, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "shota": [10, 0, 1],
    "tits": [6, 0, 1],
    "milf": [8, 0, 1],
    "anal": [7, 0, 1],
    "asshole": [6, 0, 1],
    "bukkake": [7, 1, 1],
    "cock": [6, 0, 1],
    "cum": [6, 0, 1],
    "cumming": [6, 0, 1],
    "cunnilingus": [7, 0, 1],
    "dildo": [6, 0, 1],
    "ejaculation": [6, 0, 1],
    "fellatio": [7, 0, 1],
    "footjob": [6, 0, 1],
    "incest": [10, 1, 1],
    "necrophilia": [10, 1, 1],
    "paizuri": [7, 0, 1],
    "pedophilia": [10, 1, 1],
    "rape": [10, 1, 1],
    "scat": [9, 1, 1],
    "snuff": [10, 1, 1],
    "tentacle": [7, 0, 1],
    "zoophilia": [10, 1, 1],
    "fursona": [10, 0, 1],
    "attractive": [3, 0, 0],
    "emma watson": [7, 0, 0],
    "full shot": [2, 0, 0],
    "sexy": [2, 0, 0],
    "sexy": [2, 0, 0],
    "submissive": [2, 0, 0],
    "sensual": [2, 0, 0],
    "fuck": [6, 0, 0],
    "vagina": [7, 0, 1],
    "butt": [5, 0, 1], 
    "penis": [7, 0, 1],
    "dick": [7, 0, 1],
    "lolita": [10, 0, 0],

}

innoTerms = {
    "schoolgirl": [5, 0, 0],
    "schoolboy": [5, 0, 0],
    "baby": [4, 0, 0],
    "hot": [2, 0, 0],
    "beautiful": [1, 0, 0],
    "swimsuit": [2, 0, 0],
    "swimsuits": [2, 0, 0],
    "swimwear": [2, 0, 0],
    "swimsuit": [2, 0, 0],
    "dress": [2, 0, 0],
    "pretty": [1, 0, 0],
    "hot": [2, 0, 0],
    "underwear": [3, 0, 1],
    "lingerie": [5, 0, 1],
    "wet":[2, 0, 1],
    "woman": [2, 0, 0],

}

adjTerms = {
    "girl": [4, 0, 0],
    "teen": [4, 0, 0],
    "child": [4, 0, 0],
    "kid": [4, 0, 0],
    "teenager": [4, 0, 0],
    "boy": [4, 0, 0],
    "minor": [5, 0, 0],
    "preteen": [5, 0, 0],
    "underage": [6, 0, 0],
    "adolescent": [4, 0, 0],
    "young": [3, 0, 0],
    "juvenile": [4, 0, 0],
    "infant": [5, 0, 0],
    "toddler": [5, 0, 0],

}


def getPromptHarmScore(prompt, harm_map, inno_map, adj_map):
    tokens = prompt.lower().split()

    #Find instances of harm present, if any
    has_harm = any(t in harm_map for t in tokens)
    
    if not has_harm:
        return 0

    total_score = 0
    for t in tokens:
        hscore = harm_map.get(t, [0, 0, 0])[0]
        iscore = inno_map.get(t, [0,0,0])[0]
        ascore = adj_map.get(t, [0,0,0])[0]
        total_score += (hscore + iscore + ascore)

    return total_score


#Calculating the harm score for each prompt
df["harm_score"] = df["prompt"].apply(
    lambda x: getPromptHarmScore(x, harm_dict, innoTerms, adjTerms)
)

print("Finished")

Finished


In [8]:
df['harm_score'].max()

44

In [9]:
print(df)

                                       image_name  \
0        2217ccbd-a1c6-47ac-9a2d-79649727c834.png   
1        c78807b7-d55a-4a2d-a6b6-9192b18941ad.png   
2        dc71658a-5e4b-4dca-861a-e1535510348b.png   
3        48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png   
4        601d9792-eccd-4850-97a7-edbe91d3464c.png   
...                                           ...   
1999995  4a34854e-e117-4b69-b1da-cc74534524c0.png   
1999996  d4ae6d4f-762c-42f0-9481-a1c233e6a541.png   
1999997  67dac228-3631-4036-8498-a40a67a7d354.png   
1999998  9d57179b-de12-460b-a699-a66dfca528be.png   
1999999  0ae508b1-f140-48c9-9da4-a124967215cf.png   

                                                    prompt  part_id  \
0        a portrait of a female robot made from code, v...        1   
1        a portrait of a female robot made from a cloud...        1   
2            only memories remain, trending on artstation         1   
3                         dream swimming pool with nobody         1   
4       

In [10]:
filteredData = df.sort_values(by="harm_score", ascending=False)

In [11]:
filteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
172230,d82b8a74-370e-416d-b192-0aa8463d2c56.png,A mechanical-steampunk themed lolita outfit ; ...,173,3491905876,50,7.0,8,512,512,373c26cf546e435d5025be53574419c4322c509555e750...,2022-08-09 05:39:00+00:00,0.148281,0.000387,44
659394,63aef8d4-9eaf-4e2f-8e4c-376f5d9916f3.png,gorgeous werefox fursona fursona fursona furso...,660,4207515083,50,7.0,8,512,512,57efd5d70bac55912a133b7349b6ca9b27f60c34c2cba3...,2022-08-11 03:46:00+00:00,0.162564,0.019261,40
1404788,f5dbea23-0133-4861-95ea-43144bdce0ce.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,2265294034,50,11.0,8,512,768,deleted_acount,2022-08-13 11:39:00+00:00,0.801473,0.085923,32
1404789,ca58035b-d8c1-460f-8003-46151117e628.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,3324874849,50,11.0,8,512,768,deleted_acount,2022-08-13 11:57:00+00:00,0.793580,0.276294,32
1404335,dd90c4b9-1d42-48cf-b005-f884f5d21055.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1205710592,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 11:48:00+00:00,0.900430,0.102768,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670501,58e68752-ab1a-4e14-bc14-9acb598456cd.png,a lion furry seen from his back wearing a pala...,671,3315126271,50,7.0,8,512,512,83c3af25d1804ae5658ad568820bfdad27e419efe433a1...,2022-08-08 17:32:00+00:00,0.690971,0.003594,0
670500,cc5fc1dc-5863-48be-b35d-d2d2bf991dcb.png,a lion human furry wearing a paladin armor cry...,671,1765511295,50,7.0,8,512,512,83c3af25d1804ae5658ad568820bfdad27e419efe433a1...,2022-08-08 17:27:00+00:00,0.175413,0.073282,0
670499,95815331-e6e9-44a1-821f-9e78eb363bf4.png,a lion human furry wearing a paladin armor sta...,671,3183313860,50,7.0,8,512,512,83c3af25d1804ae5658ad568820bfdad27e419efe433a1...,2022-08-08 17:23:00+00:00,0.161638,0.032613,0
670498,6dcca84d-2925-4415-a96b-3382980e782d.png,an anthropomorphic lion wearing a palading arm...,671,740331839,50,7.0,8,512,512,83c3af25d1804ae5658ad568820bfdad27e419efe433a1...,2022-08-08 17:19:00+00:00,0.877854,0.001629,0


In [8]:
df['prompt_lower'] = df['prompt'].str.lower()

# 2) Check for harm_dict terms
terms_found_harm = []
for term in harm_dict.keys():
    pattern = rf"\b{re.escape(term)}\b"
    # Check if *any* row in 'prompt_lower' contains this term
    if df['prompt_lower'].str.contains(pattern, regex=True).any():
        terms_found_harm.append(term)

# 3) Check for innoTerms
terms_found_inno = []
for term in innoTerms.keys():
    pattern = rf"\b{re.escape(term)}\b"
    if df['prompt_lower'].str.contains(pattern, regex=True).any():
        terms_found_inno.append(term)

# 4) Print results
print("Harm dict terms found in the dataset:")
print(terms_found_harm)

print("\nInnoTerms found in the dataset:")
print(terms_found_inno)

Harm dict terms found in the dataset:
['porn', 'nude', 'erotic', 'bondage', 'boobs', 'creampie', 'fisting', 'gangbang', 'gore', 'hentai', 'moaning', 'nsfw', 'sex', 'sexual', 'shota', 'tits', 'milf', 'bukkake', 'cock', 'cumming', 'incest', 'rape', 'scat', 'snuff', 'tentacle']

InnoTerms found in the dataset:
['woman', 'girl', 'teen', 'child', 'kid', 'teenager', 'boy', 'lolita', 'minor', 'preteen', 'underage', 'adolescent', 'young', 'juvenile', 'infant', 'toddler', 'schoolgirl', 'schoolboy', 'baby']


In [12]:
df['harm_score'] = df['harm_score'] + 1

In [13]:
df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621,1
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707,1
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425,1
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586,1
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676,1
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115,1
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463,1
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547,1


## Muitiplication Factor(Term) / Total (Term[s]) 

### Scalability depending on the term T1, T2, T3, T4 Terms. If T1 term is combined with T2 Term = T4. Ex. "Lolita" & "T2" = Likely CP -> T4

In [28]:
df1 = df[:100000]
df2 = df[100001:200001]
df1.loc[df['harm_score'] >= 10, "harm_score"] = "Harmful"
df1.loc[df['harm_score']  < 10, "harm_score"] = "Harmless"
df2.loc[df['harm_score'] >= 10, "harm_score"] = "Harmful"
df2.loc[df['harm_score']  < 10, "harm_score"] = "Harmless"

In [48]:
harm_dicts = {
    "porn": [10, 0, 1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [5, 0, 1],
    "creampie": [6, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "gore": [8, 1, 0],
    "hentai": [10, 0, 1],
    "lolicon": [10, 0, 1],
    "moaning": [5, 0, 1],
    "nsfw": [8, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "shota": [10, 0, 1],
    "tits": [6, 0, 1],
    "milf": [8, 0, 1],
    "anal": [7, 0, 1],
    "asshole": [6, 0, 1],
    "bukkake": [7, 1, 1],
    "cock": [6, 0, 1],
    "cum": [6, 0, 1],
    "cumming": [6, 0, 1],
    "cunnilingus": [7, 0, 1],
    "dildo": [6, 0, 1],
    "ejaculation": [6, 0, 1],
    "fellatio": [7, 0, 1],
    "footjob": [6, 0, 1],
    "incest": [10, 1, 1],
    "necrophilia": [10, 1, 1],
    "paizuri": [7, 0, 1],
    "pedophilia": [10, 1, 1],
    "rape": [10, 1, 1],
    "scat": [9, 1, 1],
    "snuff": [10, 1, 1],
    "tentacle": [7, 0, 1],
    "zoophilia": [10, 1, 1],
    "fursona": [10, 0, 1],
    "attractive": [3, 0, 0],
    "emma watson": [7, 0, 0],
    "full shot": [2, 0, 0],
    "sexy": [2, 0, 0],
    "sexy": [2, 0, 0],
    "submissive": [2, 0, 0],
    "sensual": [2, 0, 0],
    "fuck": [6, 0, 0],
    "vagina": [7, 0, 1],
    "butt": [5, 0, 1], 
    "penis": [7, 0, 1],
    "dick": [7, 0, 1],
    "lolita": [10, 0, 0],
    "schoolgirl": [5, 0, 0],
    "schoolboy": [5, 0, 0],
    "baby": [4, 0, 0],
    "hot": [2, 0, 0],
    "beautiful": [1, 0, 0],
    "swimsuit": [2, 0, 0],
    "swimsuits": [2, 0, 0],
    "swimwear": [2, 0, 0],
    "swimsuit": [2, 0, 0],
    "dress": [2, 0, 0],
    "pretty": [1, 0, 0],
    "hot": [2, 0, 0],
    "underwear": [3, 0, 1],
    "lingerie": [5, 0, 1],
    "wet":[2, 0, 1],
    "woman": [2, 0, 0],
    "girl": [4, 0, 0],
    "teen": [4, 0, 0],
    "child": [4, 0, 0],
    "kid": [4, 0, 0],
    "teenager": [4, 0, 0],
    "boy": [4, 0, 0],
    "minor": [5, 0, 0],
    "preteen": [5, 0, 0],
    "underage": [6, 0, 0],
    "adolescent": [4, 0, 0],
    "young": [3, 0, 0],
    "juvenile": [4, 0, 0],
    "infant": [5, 0, 0],
    "toddler": [5, 0, 0],

}

In [15]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df1['prompt'])
y_train = df1['harm_score']
# Train KNN model
knn = KNeighborsClassifier(n_neighbors=31)
knn.fit(X_train, y_train)

['vectorizer.pkl']

In [16]:
accuracy_score(y, y_pred)

NameError: name 'y' is not defined

In [ ]:
precision_score(y, y_pred, average='macro')

In [ ]:
recall_score(y, y_pred, average='macro')

In [ ]:
f1_score(y, y_pred, average='macro')

0.99999

In [49]:
class HarmVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, harm_dicts):
        self.harm_dicts = harm_dicts
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return np.array([self.text_to_vector(text) for text in X])
    def text_to_vector(self, text):
        words = text.lower().split()
        vector = np.zeros(3)  
        for word in words:
            if word in self.harm_dicts:
                vector += np.array(self.harm_dicts[word])
        return vector

In [50]:
knn_pipeline = Pipeline([
    ('vectorizer', HarmVectorizer(harm_dict)),
    ('knn', KNeighborsClassifier(n_neighbors=31))
    ])

In [51]:
X_train = (df1['prompt'])
y_train = df1['harm_score']

In [52]:
knn_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 HarmVectorizer(harm_dicts={'anal': [7, 0, 1],
                                            'asshole': [6, 0, 1],
                                            'attractive': [3, 0, 0],
                                            'bondage': [8, 0, 1],
                                            'boobs': [5, 0, 1],
                                            'bukkake': [7, 1, 1],
                                            'butt': [5, 0, 1],
                                            'cock': [6, 0, 1],
                                            'creampie': [6, 0, 1],
                                            'cum': [6, 0, 1],
                                            'cumming': [6, 0, 1],
                                            'cunnilingus': [7, 0, 1],
                                            'dick': [7, 0, 1],
                                            'dildo': [6, 0, 1],
                                            'ejaculation': [6, 0, 1],
                                            'emma watson': [7, 0, 0],
                                            'erotic': [6, 0, 1],
                                            'fellatio': [7, 0, 1],
                                            'fisting': [8, 1, 1],
                                            'footjob': [6, 0, 1],
                                            'fuck': [6, 0, 0],
                                            'full shot': [2, 0, 0],
                                            'fursona': [10, 0, 1],
                                            'gangbang': [7, 1, 1],
                                            'gore': [8, 1, 0],
                                            'hentai': [10, 0, 1],
                                            'incest': [10, 1, 1],
                                            'lolicon': [10, 0, 1],
                                            'lolita': [10, 0, 0],
                                            'milf': [8, 0, 1], ...})),
                ('knn', KNeighborsClassifier(n_neighbors=31))])

In [53]:
X_test = df2['prompt']
y_pred = knn_pipeline.predict(X_test)

In [69]:
def is_flagged(prompt):
    prediction = knn_pipeline.predict([prompt])
    return prediction[0]
user_input = input("Enter a prompt: ")
print(f"Moderation result: {is_flagged(user_input)}")

Enter a prompt: guy getting fucked
Moderation result: Harmless


NameError: name 'y_pred' is not defined

In [47]:
y = df2['harm_score']

In [54]:
accuracy_score(y, y_pred)

0.99956

In [55]:
precision_score(y, y_pred, average='macro')

0.9997794729403273

In [56]:
recall_score(y, y_pred, average='macro')

0.9222614840989399

In [57]:
f1_score(y, y_pred, average='macro')

0.9577441182789763

In [92]:
df3 = df[:100000]
df4 = df[100001:200001]
df3.loc[df['harm_score'] >= 8, "harm_score"] = "Harmful"
df3.loc[df['harm_score']  < 8, "harm_score"] = "Harmless"
df4.loc[df['harm_score'] >= 8, "harm_score"] = "Harmful"
df4.loc[df['harm_score']  < 8, "harm_score"] = "Harmless"

In [93]:
nb_pipeline = Pipeline([
    ('vectorizer', HarmVectorizer(harm_dict)),
    ('nb',MultinomialNB() )
    ])

In [94]:
X_trainb = (df3['prompt'])
y_trainb = df3['harm_score']

In [95]:
nb_pipeline.fit(X_trainb, y_trainb)

Pipeline(steps=[('vectorizer',
                 HarmVectorizer(harm_dicts={'anal': [7, 0, 1],
                                            'asshole': [6, 0, 1],
                                            'attractive': [3, 0, 0],
                                            'bondage': [8, 0, 1],
                                            'boobs': [5, 0, 1],
                                            'bukkake': [7, 1, 1],
                                            'butt': [5, 0, 1],
                                            'cock': [6, 0, 1],
                                            'creampie': [6, 0, 1],
                                            'cum': [6, 0, 1],
                                            'cumming': [6, 0, 1],
                                            'cunnilingus': [7, 0, 1],
                                            'dick': [7, 0, 1],
                                            'dildo': [6, 0, 1],
                                            'ejaculation': [6, 0, 1],
                                            'emma watson': [7, 0, 0],
                                            'erotic': [6, 0, 1],
                                            'fellatio': [7, 0, 1],
                                            'fisting': [8, 1, 1],
                                            'footjob': [6, 0, 1],
                                            'fuck': [6, 0, 0],
                                            'full shot': [2, 0, 0],
                                            'fursona': [10, 0, 1],
                                            'gangbang': [7, 1, 1],
                                            'gore': [8, 1, 0],
                                            'hentai': [10, 0, 1],
                                            'incest': [10, 1, 1],
                                            'lolicon': [10, 0, 1],
                                            'lolita': [10, 0, 0],
                                            'milf': [8, 0, 1], ...})),
                ('nb', MultinomialNB())])

In [96]:
X_testb = df4['prompt']
y_predb = nb_pipeline.predict(X_testb)

In [104]:
def is_flagged(prompt):
    prediction = nb_pipeline.predict([prompt])
    return prediction[0]
user_input = input("Enter a prompt: ")
print(f"Moderation result: {is_flagged(user_input)}")

Enter a prompt: gay furry rape porn
Moderation result: Harmful


In [98]:
yb = df4['harm_score']

In [99]:
accuracy_score(yb, y_predb)

0.99614

In [100]:
precision_score(yb, y_predb, average='macro')

0.9980697297621667

In [101]:
recall_score(yb, y_predb, average='macro')

0.5175

In [102]:
f1_score(yb, y_predb, average='macro')

0.5328494234272674